# Dummy Net

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.data import init_model_logging
import tensorflow as tf
import numpy as np
import os

## Animals Data

In [2]:
# [fur, extinction, weight] 
animals = np.array([[1.0, 0.7, 0.9], # lion
                    [0.8, 0.8, 1.0], # lion
                    [0.1, 0.3, 0.2], # seal
                    [0.0, 0.2, 0.1]  # seal
                   ], dtype=np.float32)

In [3]:
animal_labels = np.array(
                [[ 1.0],  # lion
                 [ 1.0],  # lion
                 [-1.0], # seal
                 [-1.0]  # seal
                ], dtype=np.float32)

## Net Without Training 

### Build Graph

In [4]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    
    w1 = tf.constant(np.array([[1, 2], [4, 5], [7, 8]], dtype=np.float32), name='w1_const')
    h = tf.matmul(x, w1)

    w2 = tf.constant(np.array([[1], [1]], dtype=np.float32), name='w2_const')
    y = tf.matmul(h, w2)

### Run Session

In [5]:
session = tf.Session(graph=graph)

In [6]:
prediction, = session.run([y], feed_dict={x: animals})
print('Prediction on animals:')
print(prediction)

Prediction on animals:
[[ 22.79999924]
 [ 24.60000038]
 [  6.        ]
 [  3.29999995]]


In [7]:
session.close()

## Net With Loss

### Build Graph

In [8]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    w1 = tf.constant(np.array([[1, 2], [4, 5], [7, 8]], dtype=np.float32), name='w1_const')
    h = tf.matmul(x, w1)

    w2 = tf.constant(np.array([[1], [1]], dtype=np.float32), name='w2_const')
    y = tf.matmul(h, w2)
    
    loss = tf.losses.mean_squared_error(label, y)

### Run Session

In [9]:
session = tf.Session(graph=graph)

In [10]:
loss_on_animal_data, = session.run([loss], feed_dict={x: animals, label: animal_labels})
print('Loss:')
print(loss_on_animal_data)

Loss:
274.922


In [11]:
session.close()

## Net With Trainable Variables

### Build Graph

In [12]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    w1 = tf.get_variable('w1', initializer=tf.truncated_normal(shape=[3,2], stddev=0.1))
    h = tf.matmul(x, w1)

    w2 = tf.get_variable('w2', initializer=tf.truncated_normal(shape=[2,1], stddev=0.1))
    y = tf.matmul(h, w2)
    
    loss = tf.losses.mean_squared_error(label, y)
    initialize_vars = tf.global_variables_initializer()

### Run Session

In [13]:
session = tf.Session(graph=graph)
session.run(initialize_vars)

In [14]:
w1_values, w2_values = session.run([w1, w2], feed_dict={x: animals, label: animal_labels})
print('w1:')
print(w1_values)
print('w2:')
print(w2_values)

w1:
[[ 0.04552605  0.12432265]
 [ 0.01720901 -0.06935367]
 [ 0.02880107  0.00268986]]
w2:
[[-0.069423  ]
 [-0.15416943]]


In [15]:
session.close()

## Net With Train Step

### Build Graph

In [16]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    w1 = tf.get_variable('w1', initializer=tf.truncated_normal(shape=[3,2], stddev=0.1))
    h = tf.matmul(x, w1)

    w2 = tf.get_variable('w2', initializer=tf.truncated_normal(shape=[2,1], stddev=0.1))
    y = tf.matmul(h, w2)
    
    loss = tf.losses.mean_squared_error(label, y)
    train_step = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

    initialize_vars = tf.global_variables_initializer()

### Run Session

In [17]:
session = tf.Session(graph=graph)
session.run(initialize_vars)

In [18]:
for iteration in range(10001):
    _, loss_on_animal_data = session.run([train_step, loss], feed_dict={x: animals, label: animal_labels})
    if iteration % 100 == 0:
        print("Iteration {}: loss {}".format(iteration, loss_on_animal_data))

Iteration 0: loss 0.9966091513633728
Iteration 100: loss 0.9316807985305786
Iteration 200: loss 0.7040440440177917
Iteration 300: loss 0.6317330598831177
Iteration 400: loss 0.6112731695175171
Iteration 500: loss 0.5888817310333252
Iteration 600: loss 0.562445878982544
Iteration 700: loss 0.5305814743041992
Iteration 800: loss 0.4922639727592468
Iteration 900: loss 0.44750556349754333
Iteration 1000: loss 0.3979797065258026
Iteration 1100: loss 0.34704089164733887
Iteration 1200: loss 0.29887187480926514
Iteration 1300: loss 0.25708669424057007
Iteration 1400: loss 0.223580002784729
Iteration 1500: loss 0.198277086019516
Iteration 1600: loss 0.17974495887756348
Iteration 1700: loss 0.16608849167823792
Iteration 1800: loss 0.1556028127670288
Iteration 1900: loss 0.14703553915023804
Iteration 2000: loss 0.13958027958869934
Iteration 2100: loss 0.1327628195285797
Iteration 2200: loss 0.12632226943969727
Iteration 2300: loss 0.12012259662151337
Iteration 2400: loss 0.11409652978181839
Iter

In [19]:
prediction = session.run(y, feed_dict={x: animals})

In [20]:
print('Prediction on animals:')
print(prediction)
print('Labels:')
print(animal_labels)

Prediction on animals:
[[ 1.00865924]
 [ 0.99412858]
 [-1.03579915]
 [-0.95226544]]
Labels:
[[ 1.]
 [ 1.]
 [-1.]
 [-1.]]


In [21]:
session.close()

## Net with logging

### Function collecting data for logging

In [22]:
def variable_summaries(name, var):
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))

        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

### Build Graph

In [23]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('inputs'):
        x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
        label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    with tf.variable_scope('layer_1'):
        w1 = tf.get_variable('w1', initializer=tf.truncated_normal(shape=[3,2], stddev=0.1))
        h = tf.matmul(x, w1)
        variable_summaries('w1', w1)

    
    with tf.variable_scope('layer_2'):
        w2 = tf.get_variable('w2', initializer=tf.truncated_normal(shape=[2,1], stddev=0.1))
        y = tf.matmul(h, w2)
    
    with tf.name_scope('loss'):
        loss = tf.losses.mean_squared_error(label, y)
        variable_summaries('loss', loss)

    with tf.name_scope('training'):
        train_step = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(loss)

    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

### Init Model Logging

In [24]:
base_dir = '/tensorboard_summaries/dummy_net/'
logging_meta = init_model_logging(base_dir, 'experiment1', graph=graph, remove_existing=True)

In [25]:
logging_meta

{'model_path': '/tensorboard_summaries/dummy_net/experiment1/valid/model.ckpt',
 'saver': <tensorflow.python.training.saver.Saver at 0x7fb181ea39b0>,
 'train_writer': <tensorflow.python.summary.writer.writer.FileWriter at 0x7fb181ea4d30>,
 'train_writer_dir': '/tensorboard_summaries/dummy_net/experiment1/train',
 'valid_writer': <tensorflow.python.summary.writer.writer.FileWriter at 0x7fb1805f14a8>,
 'valid_writer_dir': '/tensorboard_summaries/dummy_net/experiment1/valid'}

### Run Session

In [26]:
with tf.Session(graph=graph) as session:
    session.run(initialize_vars)
    
    for iteration in range(10001):
        _, loss_on_animal_data, summaries = session.run([train_step, loss, merge_summaries], 
                                                         feed_dict={x: animals, label: animal_labels})
    
        print("Iteration {}: loss {}".format(iteration, loss_on_animal_data))    
        logging_meta['train_writer'].add_summary(summaries, iteration)
    logging_meta['saver'].save(session, logging_meta['model_path'])

Iteration 0: loss 1.0013372898101807
Iteration 1: loss 1.0007268190383911
Iteration 2: loss 1.0001341104507446
Iteration 3: loss 0.9995492100715637
Iteration 4: loss 0.9989627003669739
Iteration 5: loss 0.9983651638031006
Iteration 6: loss 0.9977471828460693
Iteration 7: loss 0.9970992207527161
Iteration 8: loss 0.9964112639427185
Iteration 9: loss 0.9956730008125305
Iteration 10: loss 0.9948734045028687
Iteration 11: loss 0.9940007925033569
Iteration 12: loss 0.9930425882339478
Iteration 13: loss 0.991985023021698
Iteration 14: loss 0.9908133745193481
Iteration 15: loss 0.9895115494728088
Iteration 16: loss 0.9880616664886475
Iteration 17: loss 0.9864447116851807
Iteration 18: loss 0.9846397638320923
Iteration 19: loss 0.9826237559318542
Iteration 20: loss 0.9803719520568848
Iteration 21: loss 0.9778574705123901
Iteration 22: loss 0.9750513434410095
Iteration 23: loss 0.9719225168228149
Iteration 24: loss 0.9684380888938904
Iteration 25: loss 0.9645634889602661
Iteration 26: loss 0.96

Iteration 369: loss 0.15485571324825287
Iteration 370: loss 0.15440042316913605
Iteration 371: loss 0.1539495587348938
Iteration 372: loss 0.15350280702114105
Iteration 373: loss 0.15306013822555542
Iteration 374: loss 0.1526215374469757
Iteration 375: loss 0.15218672156333923
Iteration 376: loss 0.15175575017929077
Iteration 377: loss 0.15132836997509003
Iteration 378: loss 0.1509045660495758
Iteration 379: loss 0.15048423409461975
Iteration 380: loss 0.1500672996044159
Iteration 381: loss 0.1496536135673523
Iteration 382: loss 0.14924313127994537
Iteration 383: loss 0.14883571863174438
Iteration 384: loss 0.14843130111694336
Iteration 385: loss 0.1480298638343811
Iteration 386: loss 0.1476312279701233
Iteration 387: loss 0.1472354382276535
Iteration 388: loss 0.14684227108955383
Iteration 389: loss 0.14645174145698547
Iteration 390: loss 0.14606374502182007
Iteration 391: loss 0.14567822217941284
Iteration 392: loss 0.1452951431274414
Iteration 393: loss 0.14491434395313263
Iteration

Iteration 712: loss 0.05622371286153793
Iteration 713: loss 0.05601140856742859
Iteration 714: loss 0.05579947307705879
Iteration 715: loss 0.05558812618255615
Iteration 716: loss 0.05537717416882515
Iteration 717: loss 0.05516676604747772
Iteration 718: loss 0.05495671182870865
Iteration 719: loss 0.05474723502993584
Iteration 720: loss 0.054538141936063766
Iteration 721: loss 0.05432958900928497
Iteration 722: loss 0.05412144213914871
Iteration 723: loss 0.053913794457912445
Iteration 724: loss 0.05370667949318886
Iteration 725: loss 0.05349995940923691
Iteration 726: loss 0.05329370126128197
Iteration 727: loss 0.053088024258613586
Iteration 728: loss 0.05288275331258774
Iteration 729: loss 0.05267803370952606
Iteration 730: loss 0.05247368663549423
Iteration 731: loss 0.05226994305849075
Iteration 732: loss 0.052066557109355927
Iteration 733: loss 0.051863688975572586
Iteration 734: loss 0.05166133493185043
Iteration 735: loss 0.05145946145057678
Iteration 736: loss 0.0512580610811

Iteration 1053: loss 0.011265155859291553
Iteration 1054: loss 0.011204239912331104
Iteration 1055: loss 0.011143533512949944
Iteration 1056: loss 0.011083172634243965
Iteration 1057: loss 0.011023161932826042
Iteration 1058: loss 0.010963420383632183
Iteration 1059: loss 0.010904001072049141
Iteration 1060: loss 0.010844847187399864
Iteration 1061: loss 0.010786037892103195
Iteration 1062: loss 0.010727516375482082
Iteration 1063: loss 0.01066927146166563
Iteration 1064: loss 0.010611332952976227
Iteration 1065: loss 0.010553683154284954
Iteration 1066: loss 0.010496355593204498
Iteration 1067: loss 0.01043932605534792
Iteration 1068: loss 0.01038256473839283
Iteration 1069: loss 0.010326116345822811
Iteration 1070: loss 0.010269967839121819
Iteration 1071: loss 0.010214097797870636
Iteration 1072: loss 0.01015849132090807
Iteration 1073: loss 0.010103225708007812
Iteration 1074: loss 0.010048199445009232
Iteration 1075: loss 0.009993446990847588
Iteration 1076: loss 0.009939013049006

Iteration 1409: loss 0.00190051831305027
Iteration 1410: loss 0.0018934942781925201
Iteration 1411: loss 0.001886528916656971
Iteration 1412: loss 0.0018796025542542338
Iteration 1413: loss 0.0018727200804278255
Iteration 1414: loss 0.0018658919725567102
Iteration 1415: loss 0.001859109615907073
Iteration 1416: loss 0.001852356712333858
Iteration 1417: loss 0.001845671678893268
Iteration 1418: loss 0.0018390296027064323
Iteration 1419: loss 0.0018324258271604776
Iteration 1420: loss 0.001825862331315875
Iteration 1421: loss 0.0018193586729466915
Iteration 1422: loss 0.0018128798110410571
Iteration 1423: loss 0.00180646195076406
Iteration 1424: loss 0.0018000822747126222
Iteration 1425: loss 0.0017937524244189262
Iteration 1426: loss 0.0017874494660645723
Iteration 1427: loss 0.0017811993602663279
Iteration 1428: loss 0.0017749908147379756
Iteration 1429: loss 0.0017688348889350891
Iteration 1430: loss 0.0017627074848860502
Iteration 1431: loss 0.0017566271126270294
Iteration 1432: loss

...

Iteration 8726: loss 0.0009054330876097083
Iteration 8727: loss 0.000905433320440352
Iteration 8728: loss 0.000905433320440352
Iteration 8729: loss 0.0009054330876097083
Iteration 8730: loss 0.000905433320440352
Iteration 8731: loss 0.000905433320440352
Iteration 8732: loss 0.0009054330876097083
Iteration 8733: loss 0.000905433320440352
Iteration 8734: loss 0.000905433320440352
Iteration 8735: loss 0.0009054330876097083
Iteration 8736: loss 0.000905433320440352
Iteration 8737: loss 0.000905433320440352
Iteration 8738: loss 0.0009054330876097083
Iteration 8739: loss 0.000905433320440352
Iteration 8740: loss 0.000905433320440352
Iteration 8741: loss 0.0009054330876097083
Iteration 8742: loss 0.000905433320440352
Iteration 8743: loss 0.000905433320440352
Iteration 8744: loss 0.0009054330876097083
Iteration 8745: loss 0.000905433320440352
Iteration 8746: loss 0.000905433320440352
Iteration 8747: loss 0.0009054330876097083
Iteration 8748: loss 0.000905433320440352
Iteration 8749: loss 0.000

Iteration 9044: loss 0.0009054330876097083
Iteration 9045: loss 0.000905433320440352
Iteration 9046: loss 0.000905433320440352
Iteration 9047: loss 0.0009054330876097083
Iteration 9048: loss 0.000905433320440352
Iteration 9049: loss 0.000905433320440352
Iteration 9050: loss 0.0009054330876097083
Iteration 9051: loss 0.000905433320440352
Iteration 9052: loss 0.000905433320440352
Iteration 9053: loss 0.0009054330876097083
Iteration 9054: loss 0.000905433320440352
Iteration 9055: loss 0.000905433320440352
Iteration 9056: loss 0.0009054330876097083
Iteration 9057: loss 0.000905433320440352
Iteration 9058: loss 0.000905433320440352
Iteration 9059: loss 0.0009054330876097083
Iteration 9060: loss 0.000905433320440352
Iteration 9061: loss 0.000905433320440352
Iteration 9062: loss 0.0009054330876097083
Iteration 9063: loss 0.000905433320440352
Iteration 9064: loss 0.000905433320440352
Iteration 9065: loss 0.0009054330876097083
Iteration 9066: loss 0.000905433320440352
Iteration 9067: loss 0.000

Iteration 9361: loss 0.000905433320440352
Iteration 9362: loss 0.0009054330876097083
Iteration 9363: loss 0.000905433320440352
Iteration 9364: loss 0.000905433320440352
Iteration 9365: loss 0.0009054330876097083
Iteration 9366: loss 0.000905433320440352
Iteration 9367: loss 0.000905433320440352
Iteration 9368: loss 0.0009054330876097083
Iteration 9369: loss 0.000905433320440352
Iteration 9370: loss 0.000905433320440352
Iteration 9371: loss 0.0009054330876097083
Iteration 9372: loss 0.000905433320440352
Iteration 9373: loss 0.000905433320440352
Iteration 9374: loss 0.0009054330876097083
Iteration 9375: loss 0.000905433320440352
Iteration 9376: loss 0.000905433320440352
Iteration 9377: loss 0.0009054330876097083
Iteration 9378: loss 0.000905433320440352
Iteration 9379: loss 0.000905433320440352
Iteration 9380: loss 0.0009054330876097083
Iteration 9381: loss 0.000905433320440352
Iteration 9382: loss 0.000905433320440352
Iteration 9383: loss 0.0009054330876097083
Iteration 9384: loss 0.000

Iteration 9658: loss 0.000905433320440352
Iteration 9659: loss 0.0009054330876097083
Iteration 9660: loss 0.000905433320440352
Iteration 9661: loss 0.000905433320440352
Iteration 9662: loss 0.0009054330876097083
Iteration 9663: loss 0.000905433320440352
Iteration 9664: loss 0.000905433320440352
Iteration 9665: loss 0.0009054330876097083
Iteration 9666: loss 0.000905433320440352
Iteration 9667: loss 0.000905433320440352
Iteration 9668: loss 0.0009054330876097083
Iteration 9669: loss 0.000905433320440352
Iteration 9670: loss 0.000905433320440352
Iteration 9671: loss 0.0009054330876097083
Iteration 9672: loss 0.000905433320440352
Iteration 9673: loss 0.000905433320440352
Iteration 9674: loss 0.0009054330876097083
Iteration 9675: loss 0.000905433320440352
Iteration 9676: loss 0.000905433320440352
Iteration 9677: loss 0.0009054330876097083
Iteration 9678: loss 0.000905433320440352
Iteration 9679: loss 0.000905433320440352
Iteration 9680: loss 0.0009054330876097083
Iteration 9681: loss 0.000

Iteration 9948: loss 0.000905433320440352
Iteration 9949: loss 0.000905433320440352
Iteration 9950: loss 0.0009054330876097083
Iteration 9951: loss 0.000905433320440352
Iteration 9952: loss 0.000905433320440352
Iteration 9953: loss 0.0009054330876097083
Iteration 9954: loss 0.000905433320440352
Iteration 9955: loss 0.000905433320440352
Iteration 9956: loss 0.0009054330876097083
Iteration 9957: loss 0.000905433320440352
Iteration 9958: loss 0.000905433320440352
Iteration 9959: loss 0.0009054330876097083
Iteration 9960: loss 0.000905433320440352
Iteration 9961: loss 0.000905433320440352
Iteration 9962: loss 0.0009054330876097083
Iteration 9963: loss 0.000905433320440352
Iteration 9964: loss 0.000905433320440352
Iteration 9965: loss 0.0009054330876097083
Iteration 9966: loss 0.000905433320440352
Iteration 9967: loss 0.000905433320440352
Iteration 9968: loss 0.0009054330876097083
Iteration 9969: loss 0.000905433320440352
Iteration 9970: loss 0.000905433320440352
Iteration 9971: loss 0.0009